In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 30)

In [3]:
df = pd.read_excel("C:/Users/schne/Desktop/data/nfl_player_stats_cleaned.xlsx")

In [4]:
df_qb = df[df['position_group'] == 'QB']
#df_qb = df_qb[df_qb['depth_team'] == 1]
df_qb.head()

,player_id,player_name,player_display_name,position_group,season,week,season_type,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles_lost,passing_yards_after_catch,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles_lost,receiving_yards_after_catch,receiving_first_downs,receiving_2pt_conversions,target_share,depth_team,time_index
6,00-0000722,T.Banks,Tony Banks,QB,2001,1,REG,WAS,LAC,7,14,53,0,0,0,0,0,53,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,2,1
10,00-0000865,C.Batch,Charlie Batch,QB,2001,1,REG,DET,GB,20,39,276,0,2,7,-44,1,276,0,2,9,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,1
21,00-0001361,D.Bledsoe,Drew Bledsoe,QB,2001,1,REG,NE,CIN,22,38,241,2,0,4,-17,0,241,0,3,10,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,1
24,00-0001545,T.Bouman,Todd Bouman,QB,2001,1,REG,MIN,CAR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2,1
28,00-0001823,A.Brooks,Aaron Brooks,QB,2001,1,REG,NO,BUF,18,29,209,3,0,2,-17,0,209,0,5,10,0,0,2,0,0,0,0,0,0,0,0,0,0.0,1,1


In [5]:
df_qb['cum_avg_yards'] = df_qb['passing_yards'].expanding().mean()
df_qb['cum_std_yards'] = df_qb['passing_yards'].expanding().std()

C:\Users\schne\AppData\Local\Temp\ipykernel_4408\697136595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qb['cum_avg_yards'] = df_qb['passing_yards'].expanding().mean()
C:\Users\schne\AppData\Local\Temp\ipykernel_4408\697136595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qb['cum_std_yards'] = df_qb['passing_yards'].expanding().std()


In [6]:
print(df_qb.columns)

Index(['player_id', 'player_name', 'player_display_name', 'position_group', 'season', 'week', 'season_type', 'team', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds',
       'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles_lost', 'passing_yards_after_catch', 'passing_2pt_conversions', 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles_lost', 'receiving_yards_after_catch',
       'receiving_first_downs', 'receiving_2pt_conversions', 'target_share', 'depth_team', 'time_index', 'cum_avg_yards', 'cum_std_yards'],
      dtype='object')


In [7]:
df_qb['played_flag'] = (df_qb['passing_yards'] > 0).astype(int)

C:\Users\schne\AppData\Local\Temp\ipykernel_4408\3871678930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qb['played_flag'] = (df_qb['passing_yards'] > 0).astype(int)


In [8]:
target = 'passing_yards'   # whatever you’re predicting


for stat in['attempts','completions','passing_yards','passing_tds','passing_interceptions']:
   for lag in [1, 2, 3,4,5,6]:  # use however many weeks of history you think matter
      df_qb[f'{stat}_lag{lag}'] =df_qb[stat].shift(lag)
   df_qb[f'{stat}_rolling4'] =df_qb[stat].shift(1).rolling(4).mean()
   df_qb[f'{stat}_rolling4'] = df_qb.groupby('player_id')[stat].shift(1).rolling(4).mean().reset_index(level=0, drop=True)
      

C:\Users\schne\AppData\Local\Temp\ipykernel_4408\2905596181.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qb[f'{stat}_lag{lag}'] =df_qb[stat].shift(lag)
C:\Users\schne\AppData\Local\Temp\ipykernel_4408\2905596181.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qb[f'{stat}_lag{lag}'] =df_qb[stat].shift(lag)
C:\Users\schne\AppData\Local\Temp\ipykernel_4408\2905596181.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [9]:
df_qb.head()

,player_id,player_name,player_display_name,position_group,season,week,season_type,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles_lost,passing_yards_after_catch,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles_lost,receiving_yards_after_catch,receiving_first_downs,receiving_2pt_conversions,target_share,depth_team,time_index,cum_avg_yards,cum_std_yards,played_flag,attempts_lag1,attempts_lag2,attempts_lag3,attempts_lag4,attempts_lag5,attempts_lag6,attempts_rolling4,completions_lag1,completions_lag2,completions_lag3,completions_lag4,completions_lag5,completions_lag6,completions_rolling4,passing_yards_lag1,passing_yards_lag2,passing_yards_lag3,passing_yards_lag4,passing_yards_lag5,passing_yards_lag6,passing_yards_rolling4,passing_tds_lag1,passing_tds_lag2,passing_tds_lag3,passing_tds_lag4,passing_tds_lag5,passing_tds_lag6,passing_tds_rolling4,passing_interceptions_lag1,passing_interceptions_lag2,passing_interceptions_lag3,passing_interceptions_lag4,passing_interceptions_lag5,passing_interceptions_lag6,passing_interceptions_rolling4
6,00-0000722,T.Banks,Tony Banks,QB,2001,1,REG,WAS,LAC,7,14,53,0,0,0,0,0,53,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,2,1,53.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,00-0000865,C.Batch,Charlie Batch,QB,2001,1,REG,DET,GB,20,39,276,0,2,7,-44,1,276,0,2,9,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,1,164.5,157.684812,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
21,00-0001361,D.Bledsoe,Drew Bledsoe,QB,2001,1,REG,NE,CIN,22,38,241,2,0,4,-17,0,241,0,3,10,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,1,190.0,119.929146,1,39.0,14.0,NaN,NaN,NaN,NaN,NaN,20.0,7.0,NaN,NaN,NaN,NaN,NaN,276.0,53.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN
24,00-0001545,T.Bouman,Todd Bouman,QB,2001,1,REG,MIN,CAR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2,1,142.5,136.431912,0,38.0,39.0,14.0,NaN,NaN,NaN,NaN,22.0,20.0,7.0,NaN,NaN,NaN,NaN,241.0,276.0,53.0,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,2.0,0.0,NaN,NaN,NaN,NaN
28,00-0001823,A.Brooks,Aaron Brooks,QB,2001,1,REG,NO,BUF,18,29,209,3,0,2,-17,0,209,0,5,10,0,0,2,0,0,0,0,0,0,0,0,0,0.0,1,1,155.8,121.838828,1,0.0,38.0,39.0,14.0,NaN,NaN,NaN,0.0,22.0,20.0,7.0,NaN,NaN,NaN,0.0,241.0,276.0,53.0,NaN,NaN,NaN,0.0,2.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2.0,0.0,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# -------------------------------
# Base regression function
# -------------------------------

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def run_xgb_model(train, test, target, features):
    """
    Train XGBRegressor on non-zero target data.
    """
    X_train, y_train = train[features], train[target]
    X_test, y_test   = test[features], test[target]

    model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    # Safe MAPE
    def safe_mape(y_true, y_pred, eps=1e-8):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100

    # Metrics
    mae  = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2   = r2_score(y_test, preds)
    mask = y_test > 50
    mape = safe_mape(y_test[mask], preds[mask])
    mape = mean_absolute_error(y_test[mask], preds[mask]) / (np.mean(y_test[mask]) + 1e-8) * 100

    print(f"Regression — MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.3f} | SAFE MAPE: {mape:.2f}%, | MAPE: {mape:.2f}%)")
    return model, preds

# -------------------------------
# Two-step zero-inflated model
# -------------------------------
def run_two_step_xgb(df, target, season_split=2024, extra_features=None):
    """
    df: full DataFrame with all seasons
    target: 'rushing_yards' or similar
    season_split: season used to split train/test
    """
    # Time-based train/test split
    train = df[df['season'] < season_split].copy()
    test  = df[df['season'] == season_split].copy()

    # Step 1: Classification target
    train['played_flag'] = (train[target] > 0).astype(int)
    test['played_flag']  = (test[target] > 0).astype(int)

    # Features
    features = [col for col in df.columns if 'lag' in col or 'rolling' in col]
    if extra_features:
        features += extra_features

    # ---------------- Classification ----------------
    clf = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:pseudohubererror',
        random_state=42
    )
    clf.fit(train[features], train['played_flag'])

    # Predict probability player will get >0 yards
    prob_play = clf.predict_proba(test[features])[:, 1]
    acc = accuracy_score(test['played_flag'], (prob_play > 0.5).astype(int))
    print(f"\nStep 1 — Classification Accuracy: {acc:.3f}")

    # ---------------- Regression ----------------
    train_reg = train[train[target] > 0].copy()
    reg_model, preds_reg = run_xgb_model(train_reg, test, target, features)

    # Predict yards if player plays
    predicted_yards_if_played = reg_model.predict(test[features])

    # ---------------- Combine ----------------
    final_forecast = prob_play * predicted_yards_if_played

    # Results DataFrame
    results = test.copy()
    results['prob_play'] = prob_play
    results['predicted_yards_if_played'] = predicted_yards_if_played
    results['final_forecast'] = final_forecast

    return clf, reg_model, results

# -------------------------------
# Example: Run on your QB DataFrame
# -------------------------------
# clf, reg, results = run_two_step_xgb(df_qb, target='rushing_yards', season_split=2024)

# Show top 20 forecasts
# display(results[['player_name','season','week','rushing_yards','prob_play','predicted_yards_if_played','final_forecast']].head(20))

In [11]:
# -------------------------------
# Example: Run on your QB DataFrame
# -------------------------------
extra_features = ['depth_team']
clf, reg, results = run_two_step_xgb(df_qb, target='passing_yards', season_split=2024, extra_features=extra_features)

# Show top 20 forecasts
display(results[['player_name','season','week','prob_play', 'attempts', 'completions', 'passing_yards','predicted_yards_if_played','final_forecast']].head(20))


Step 1 — Classification Accuracy: 1.000


KeyError: "['predicted_yards_if_played', 'final_forecast'] not in index"

In [ ]:
import matplotlib.pyplot as plt
import random

# Pick 4 random player IDs that exist in your test results
sample_players = random.sample(list(results['player_id'].unique()), 6)

# Create a 2x2 subplot layout
fig, axes = plt.subplots(3, 2, figsize=(12, 8))
axes = axes.flatten()

for i, player_id in enumerate(sample_players):
    player_data = results[results['player_id'] == player_id].sort_values('week')
    
    # Get player name safely (in case of duplicates or missing values)
    player_name = player_data['player_name'].iloc[0] if 'player_name' in player_data.columns else str(player_id)

    axes[i].plot(player_data['week'], player_data['passing_yards'], 
                 label='Actual', marker='o')
    axes[i].plot(player_data['week'], player_data['final_forecast'], 
                 label='Forecast', marker='x')
    
    axes[i].set_title(f'{player_name} — Passing Yard Forecast')
    axes[i].set_xlabel('Week')
    axes[i].set_ylabel('Passing Yards')
    axes[i].legend()
    axes[i].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
results_table = results[['player_name', 'position_group', 'season', 'week', 'attempts', 'completions',
                         'passing_yards', 'prob_play', 
                         'predicted_yards_if_played', 'final_forecast']]
pd.set_option('display.max_rows', None)
# Display all rows (or use .head(100) for first 100)
results_table.head(658)

In [ ]:
extra_features = ['depth_team']
clf, reg, results = run_two_step_xgb(df_qb, target='passing_tds', season_split=2024, extra_features=extra_features)

# Show top 20 forecasts
display(results[['player_name','season','week','prob_play', 'attempts', 'completions', 'passing_yards','predicted_yards_if_played','final_forecast']].head(20))

In [ ]:

def forecast_future_weeks(df, clf, reg_model, target, features, base_season=2024, future_weeks=4):
    """
    Forecast multiple future weeks using the trained classifier and regressor.
    - df: your full dataframe (df_qb)
    - clf: trained XGBClassifier
    - reg_model: trained XGBRegressor
    - target: e.g. 'passing_yards'
    - features: list of feature columns
    - base_season: last season used for training/testing (e.g., 2024)
    - future_weeks: how many weeks ahead to forecast
    """
    last_weeks = df[df['season'] == base_season].copy()
    future_projections = []

    for i in range(1, future_weeks + 1):
        # Make a copy for future week
        future = last_weeks.copy()
        future['season'] = base_season + (1 if future['week'].max() + i > 18 else 0)
        future['week'] = future['week'] + i

        # Predict play probability and yards
        future_prob_play = clf.predict_proba(future[features])[:, 1]
        future_pred_yards = reg_model.predict(future[features])
        future_forecast = future_prob_play * future_pred_yards

        future['prob_play'] = future_prob_play
        future['predicted_yards_if_played'] = future_pred_yards
        future['final_forecast'] = future_forecast
        future['forecast_week_ahead'] = i
        future['type'] = 'future_projection'

        future_projections.append(future)

    # Combine all future projections
    future_all = pd.concat(future_projections, ignore_index=True)
    return future_all



features = [col for col in df_qb.columns if 'lag' in col or 'rolling' in col] + ['depth_team']

future_results = forecast_future_weeks(df_qb, clf, reg, 'passing_yards', features, base_season=2024, future_weeks=4)

# Combine with your test results
all_results = pd.concat([results, future_results], ignore_index=True)

print("\n=== Future Week Projections ===")
display(future_results[['player_name','season','week','prob_play','predicted_yards_if_played','final_forecast','forecast_week_ahead']].head(40))
